<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p022_other_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [2]:
pip install -q "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 4.3 MB/s 


In [3]:
import requests
import tensorflow as tf
import tensorflow_text as tf_text

# Les différents tokenizers de Tensorflow

Les inputs strings doivent être encodés en UTF-8 pour être compatibles avec les tokenizers présentés ci-dessus

___
 *  `text.WhitespaceTokenizer()`

C'est clairement le tokenizer le plus naïf, il split une phrase à chaque white space (comme les espaces, les tabulations, les sauts de ligne, etc). Le problème c'est qu'il ne distingue pas la ponctuation ce con. Par exemple : `J'aime aller au cinéma, regarder du bon vieux film américain` sera tokenisé de la façon suivante : J'aime + aller + au + cinéma, + regarder + ... Il a crée un token `cinéma,`. Il a mis une "," le con. 

In [4]:
tokenizer = tf_text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b"can't", b'explain,', b'but', b'you', b'feel', b'it.']]


____
 * `UnicodeScriptTokenizer()`

Pour parer au problème lié à la ponctuation du whitespacetokenizer() vu plus haut, il existe le `UnicodeScriptTokenizer()`. Le défaut de ce tokenizer c'est qu'il splittera le mot `can't` en trois mots `can` + `'` + `t`, ce qui n'est nécessairement pas ce qu'on souhaite.

In [5]:
tokenizer = tf_text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b'can', b"'", b't', b'explain', b',', b'but', b'you', b'feel', b'it', b'.']]


___
 * `tf_text.BertTokenizer()`

C'est l'implémentation du tokenizer utilisé dans le papier présentant BERT. En gros, il s'appuie sur le WordPiece algorithm mais rajoute quelques trucs en plus comme la lowerisation par exemple. Le tokenizer BERT nécessite l'accès au corpus dans un premier temps afin de constituer le vocabulaire

In [8]:
url = "https://github.com/tensorflow/text/blob/master/tensorflow_text/python/ops/test_data/test_wp_en_vocab.txt?raw=true"
r = requests.get(url)
filepath = "vocab.txt"
open(filepath, 'wb').write(r.content)

tokenizer = tf_text.BertTokenizer(filepath, token_out_type=tf.string, lower_case=True)
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

52382

____
 * `tf_text.SentencepieceTokenizer()`

Dans le même genre que BERT tokenizer on a le SentencepieceTokenizer, il ne rentre pas dans les détails donc je m'emmerde pas trop

In [10]:
url = "https://github.com/tensorflow/text/blob/master/tensorflow_text/python/ops/test_data/test_oss_model.model?raw=true"
sp_model = requests.get(url).content

tokenizer = tf_text.SentencepieceTokenizer(sp_model, out_type=tf.string)
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

___
 * `tf_text.UnicodeCharTokenizer()`

Dans les langues CJK, qui ne contiennent pas d'espaces, il est intéressant de tokeniser tout les caractères UTF-8. C'est ce que permet de faire `tf_text.UnicodeCharTokenizer()`. Le token de sortie est un nombre représentant un nombre représentant un caractère utf-8. On peut donc le décoder facilement avec la fonction `tf.strings.unicode_encode(tf.expand_dims(tokens, -1), "UTF-8")` pour obtenir un ragged tensor des caractères décodés.

In [12]:
tokenizer = tf_text.UnicodeCharTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())

tokens = tokenizer.tokenize(["M O A"])
print(tokens.to_list())

characters = tf.strings.unicode_encode(tf.expand_dims(tokens, -1), "UTF-8")
print(characters)

[[87, 104, 97, 116, 32, 121, 111, 117, 32, 107, 110, 111, 119, 32, 121, 111, 117, 32, 99, 97, 110, 39, 116, 32, 101, 120, 112, 108, 97, 105, 110, 44, 32, 98, 117, 116, 32, 121, 111, 117, 32, 102, 101, 101, 108, 32, 105, 116, 46]]
[[77, 32, 79, 32, 65]]


___
 * `tf_text.HubModuleTokenizer()`

Il existe des tokenizers spécifiques présents sur le Hub de Tensorflow. Notamment celui qui permet de parser du chinois, i.e. de séparer les mots entre eux alors qu'il n'existe pas d'espaces dans cette langue. Souvent, ce qu'on fait, c'est qu'on crée des fonctions de décodage et encodage UTF-8 pour lire convenablement les caractères

In [15]:
MODEL_HANDLE = "https://tfhub.dev/google/zh_segmentation/1"
segmenter = tf_text.HubModuleTokenizer(MODEL_HANDLE)
tokens = segmenter.tokenize(["新华社北京"])
print(tokens.to_list())

# Un poil énervé la récursivité
def decode_list(x):
  if type(x) is list:
    return list(map(decode_list, x))
  return x.decode("UTF-8")
def decode_utf8_tensor(x):
  return list(map(decode_list, x.to_list()))

print(decode_utf8_tensor(tokens))

[[b'\xe6\x96\xb0\xe5\x8d\x8e\xe7\xa4\xbe', b'\xe5\x8c\x97\xe4\xba\xac']]
[['新华社', '北京']]


___
 * `tf_text.SplitMergeTokenizer()`

On peut vouloir tokenizer un corpus en connaissant l'exact position des tokens. A ce moment là, SplitMergeTokenizer est un excellent choix. Il prend en input en plus des strings, un tableau de label qui indique le début d'un nouveau mot, représenté par 0. Le 1 indique que le caractère fait partie du mot dernièrement commencé. On peut également utilisé `tf_text.SplitMergeFromLogitsTokenizer()` qui prendrait des paires de logits issues d'un neural network pour qualifier le début ou pas de chaque mot mais j'ai rien compris.

In [16]:
strings = ["新华社北京"]
labels = [[0, 1, 1, 0, 1]]
tokenizer = tf_text.SplitMergeTokenizer()
tokens = tokenizer.tokenize(strings, labels)
print(decode_utf8_tensor(tokens))

strings = [["新华社北京"]]
labels = [[[5.0, -3.2], [0.2, 12.0], [0.0, 11.0], [2.2, -1.0], [-3.0, 3.0]]]
tokenizer = tf_text.SplitMergeFromLogitsTokenizer()
tokenizer.tokenize(strings, labels)
print(decode_utf8_tensor(tokens))

[['新华社', '北京']]
[['新华社', '北京']]


____
 * `tf_text.RegexSplitter()`

Permet de splitter une phrase à chaque regex pattern extraits de la phrase en question. 

In [17]:
splitter = tf_text.RegexSplitter("\s")
tokens = splitter.split(["What you know you can't explain, but you feel it."], )
print(tokens.to_list())

[[b'What', b'you', b'know', b'you', b"can't", b'explain,', b'but', b'you', b'feel', b'it.']]


____
____
### Offsets methods
Tout les tokenizers sont capables de renvoyer les positions de départ et de fin de chaque token sur la phrase originale

In [18]:
tokenizer = tf_text.UnicodeScriptTokenizer()
(tokens, start_offsets, end_offsets) = tokenizer.tokenize_with_offsets(['Everything not saved will be lost.'])
print(tokens.to_list())
print(start_offsets.to_list())
print(end_offsets.to_list())

[[b'Everything', b'not', b'saved', b'will', b'be', b'lost', b'.']]
[[0, 11, 15, 21, 26, 29, 33]]
[[10, 14, 20, 25, 28, 33, 34]]


____
### Detokenization methods
Tout les tokenizers proposent une méthode de detokenization afin de passer du tensor de tokens au tensor de text tokens

In [19]:
tokenizer = tf_text.UnicodeCharTokenizer()
tokens = tokenizer.tokenize(["What you know you can't explain, but you feel it."])
print(tokens.to_list())
strings = tokenizer.detokenize(tokens)
print(strings.numpy())

[[87, 104, 97, 116, 32, 121, 111, 117, 32, 107, 110, 111, 119, 32, 121, 111, 117, 32, 99, 97, 110, 39, 116, 32, 101, 120, 112, 108, 97, 105, 110, 44, 32, 98, 117, 116, 32, 121, 111, 117, 32, 102, 101, 101, 108, 32, 105, 116, 46]]
[b"What you know you can't explain, but you feel it."]


____
### Interaction avec les Tensorflow Datasets
La tokenization des strings présents dans un tensorflow dataset se fait naturellement à l'aide la méthode `.map()` comme dans l'exemple ci-dessous :

In [20]:
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = tf_text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]
